In [1]:
import pandas as pd
import numpy as np
import os as os
#import pickle
import seaborn as sns
import matplotlib.pyplot as plt

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn import metrics
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score, StratifiedKFold, KFold, RepeatedKFold
from sklearn.metrics import classification_report
from joblib import dump, load
import acessos as ac
import warnings

warnings.simplefilter(action='ignore', category=FutureWarning)

In [2]:
text_file = open(os.path.abspath('../Análise de sentimento/verbalizacoes.sql'), "r")
query = text_file.read()
text_file.close()
df = ac.df_athena_q('flavia-costa', query)

In [3]:
df.head()

,id_issue,cpf_customer,mes_chat,dt_envio_mensagem,ds_message
0,18164186,12524962490,2023-08-01,2023-08-10,"[É outro assunto, problema com recebimento de ..."
1,18064579,04175649026,2023-08-01,2023-08-07,"[Eu pedi reembolso sobre a compra com o site, ..."
2,18396641,02036562299,2023-08-01,2023-08-20,"[Mais opções, Falar de outra coisa, Perdi meu ..."
3,18208804,06977144203,2023-08-01,2023-08-11,"[Falar de outra coisa, Pix, Não dá pra cadastr..."
4,17951777,11004491590,2023-08-01,2023-08-02,"[Parcelado, 80.00, 000, Quero falar com um esp..."


In [4]:
len(df)

852883

## Aplicando modelo sentimento

In [13]:
from sklearn.neural_network import MLPClassifier


vetorizador = load('vetorizador_sentimento.joblib') 

In [14]:
X = vetorizador.transform(df['ds_message'])

In [15]:
clf_sentiment = load('modelo_sentimento.joblib') 

In [16]:
df['sent_predito_1'] = clf_sentiment.predict(X)

### Palavras chave 

In [66]:
chave_negativo = ['falta de respeito', 
'judicial',
'procon',
'jurídico',
'porra',
'merda', 
'palhaçada',
'carai',
'caralho',
'bosta',
'vocês demoram',
'voces demoram',
'vcs demoram',
'horrivel',
'banco ruim',
'péssimo',
'danos morais',
'pelo amor de deus',
'eu preciso resolver',
'atender??',
'me roubando']


In [76]:
chave_positivo = [
'ótimo',
'otimo',
'perfeito',
'muito obrigada',
'excelente',
'ótimo dia',
'maravilha',
'deus abençoe']

In [77]:
df['sent_predito_1'].value_counts()

1    596127
0    196172
2     60584
Name: sent_predito_1, dtype: int64

In [61]:
def marca_negativo(x):
    for i in chave_negativo:
        if (x.find(i) != -1):
            return 1
        else:
            return 0
   
df['flag_negativo'] = df['ds_message'].apply(marca_negativo)

In [78]:
def marca_positivo(x):
    for i in chave_positivo:
        if (x.find(i) != -1):
            return 1
        else:
            return 0
   
df['flag_positivo'] = df['ds_message'].apply(marca_positivo)

In [79]:
df['sent_predito_final'] = np.where((df['flag_negativo'] == 1) | (df['sent_predito_1'] == 0),  0, #condições negativo
                                 np.where(df['flag_positivo'] == 1,  2, #condições positivo
                                          df['sent_predito_1']))

df['sent_predito_final'].value_counts()

1    595318
0    196215
2     61350
Name: sent_predito_final, dtype: int64

### Modelo do tipo de mensagem

Gostaria de saber
Como faço
como faço
problema
não reconheço
não fiz essa compra
não reconhecida
fui roubad
fui assaltad
limite não voltou
não consegui pagar
pix não chegou
compra que não fiz
nunca comprei
quero alterar meu
quero resolver
queria que fosse resolvido
preciso resolver
quero aumentar o meu limite
queria o aumento do meu cartao
fiquei na dúvida
paguei mas não aparece
prefiro parcelar

In [23]:
clf_tipo_msg = load('modelo_tipo_mensagem.joblib')

In [24]:
df['tipo_msg_predito'] = clf_tipo_msg.predict(X)

In [25]:
df['tipo_msg_predito'].value_counts()

0    394796
2    256753
1    201334
Name: tipo_msg_predito, dtype: int64

In [26]:
df.head()

,id_issue,cpf_customer,mes_chat,dt_envio_mensagem,ds_message,sent_predito_1,flag_negativo,flag_positivo,sent_predito_final,tipo_msg_predito
0,18164186,12524962490,2023-08-01,2023-08-10,"[É outro assunto, problema com recebimento de ...",1,0,0,1,0
1,18064579,04175649026,2023-08-01,2023-08-07,"[Eu pedi reembolso sobre a compra com o site, ...",1,0,0,1,0
2,18396641,02036562299,2023-08-01,2023-08-20,"[Mais opções, Falar de outra coisa, Perdi meu ...",1,0,0,1,1
3,18208804,06977144203,2023-08-01,2023-08-11,"[Falar de outra coisa, Pix, Não dá pra cadastr...",0,0,0,0,0
4,17951777,11004491590,2023-08-01,2023-08-02,"[Parcelado, 80.00, 000, Quero falar com um esp...",1,0,0,1,2


In [80]:
df['sentimento_predito'] = np.where(df['sent_predito_final'] == 0,  '0. Negativo', 
                           np.where(df['sent_predito_final'] == 2,  '2. Positivo', '1. Neutro'))

In [42]:
df.query('sentimento_predito == "0. Negativo"')['ds_message'][27]

'[Dúvida sobre uma compra, Apareceu uma compra que eu não fiz, Pela 5° x, Eu não vou mais falar com vcs, Vcs demora 2 horas pra responder quando responde eu não vejo pq não sou vidente, Já mandei todas as informações q vcs me pedem, Estão me fazendo de otário, Ou vcs resolvem minha situação ou irei tomar medidas judiciais, 02/09/2023 APPLE.COM/BILL SAO PAULO BRA 54,90\n02/09/2023 APPLE.COM/BILL SAO PAULO BRA 54,90\n02/09/2023 APPLE.COM/BILL SAO PAULO BRA 27,90\n02/09/2023 APPLE.COM/BILL SAO PAULO BRA 54,90\n02/09/2023 APPLE.COM/BILL SAO PAULO BRA 27,90\n02/09/2023 APPLE.COM/BILL SAO PAULO BRA 4,90\n02/09/2023 APPLE.COM/BILL SAO PAULO BRA 4,90\n27/08/2023 APPLE.COM/BILL SAO PAULO BRA 4,90, Parem de me perguntar se tem outras compras q nao foi eu q comprei q estão na minha time LINE. As únicas q não foi eu são justamente essas q te mandei, Pela 5 x, Pare de me enrolar porra, Já tô ficando estressado, Caramba rapaz, Vcs são incompetentes, Todos vcs, Sem excessão, Resolva minha questão, Ag

In [46]:
df['tipo_msg_predito'] = np.where(df['tipo_msg_predito'] == 0,  '0. Problema', 
                           np.where(df['tipo_msg_predito'] == 1,  '1. Solicitacao', '2. Duvida'))

In [59]:
df.head()

,id_issue,cpf_customer,mes_chat,dt_envio_mensagem,ds_message,sent_predito_1,flag_negativo,flag_positivo,sent_predito_final,tipo_msg_predito,sentimento_predito
0,18164186,12524962490,2023-08-01,2023-08-10,"[É outro assunto, problema com recebimento de ...",1,0,0,1,0. Problema,1. Neutro
1,18064579,04175649026,2023-08-01,2023-08-07,"[Eu pedi reembolso sobre a compra com o site, ...",1,0,0,1,0. Problema,1. Neutro
2,18396641,02036562299,2023-08-01,2023-08-20,"[Mais opções, Falar de outra coisa, Perdi meu ...",1,0,0,1,1. Solicitacao,1. Neutro
3,18208804,06977144203,2023-08-01,2023-08-11,"[Falar de outra coisa, Pix, Não dá pra cadastr...",0,0,0,0,0. Problema,0. Negativo
4,17951777,11004491590,2023-08-01,2023-08-02,"[Parcelado, 80.00, 000, Quero falar com um esp...",1,0,0,1,2. Duvida,1. Neutro


In [81]:
pd.crosstab(df.sentimento_predito, df.tipo_msg_predito, normalize='index')

tipo_msg_predito,0. Problema,1. Solicitacao,2. Duvida
sentimento_predito,,,
0. Negativo,0.669245,0.161863,0.168891
1. Neutro,0.406803,0.261039,0.332159
2. Positivo,0.347237,0.231019,0.421744


In [82]:
pd.crosstab(df.sentimento_predito, df.tipo_msg_predito, normalize='columns')

tipo_msg_predito,0. Problema,1. Solicitacao,2. Duvida
sentimento_predito,,,
0. Negativo,0.332617,0.157748,0.129070
1. Neutro,0.613423,0.771857,0.770157
2. Positivo,0.053960,0.070395,0.100774


In [51]:
df.query('sentimento_predito == "2. Positivo" and tipo_msg_predito == "0. Problema"')['ds_message']
#[117]

20                              [Sim, podemos, Sim. Recebi]
28        [Falar de outra coisa, Eu paguei o boleto pra ...
54        [Falar de outra coisa, Então eu tenho uma fatu...
106       [Pedir o reenvio, O endereço tá certo, 49100-0...
117       [Cartão físico, Entrega do cartão, Recebi hj o...
                                ...                        
852728    [Dúvida sobre uma compra, Apareceu uma compra ...
852792         [Só recebi a notificação no email da compra]
852818    [Estava correto sim. Eu moro esse portão preto...
852835    [Oii. boa tarde, Meu boleto de negociação está...
852861    [Problema com compra, Compra que não fiz, Nunc...
Name: ds_message, Length: 21870, dtype: object

In [53]:
df.query('sentimento_predito == "0. Negativo" and tipo_msg_predito == "1. Solicitacao"')['ds_message'][167]
#[117]

'[Bom dia! Eu tinha feito uma negociação com vcs para quitar o contrato, vcs foram bem claro comigo, o valor que eu paguei iria quitar os débitos vencidos e o que iria vencer, agora chegou uma fatura para eu pagar., Oi?????, Não vão me responder?]'

In [83]:
df.columns

Index(['id_issue', 'cpf_customer', 'mes_chat', 'dt_envio_mensagem',
       'ds_message', 'sent_predito_1', 'flag_negativo', 'flag_positivo',
       'sent_predito_final', 'tipo_msg_predito', 'sentimento_predito'],
      dtype='object')

In [84]:
area = 'cx'
name = 'analise_sentimento_teste.csv'
filename = f'{area}.{name}'

df[['cpf_customer', 'mes_chat', 'dt_envio_mensagem',
       'ds_message', 'sent_predito_1','tipo_msg_predito', 'sentimento_predito']].to_csv(filename, index=False)

os.system(f'aws s3 cp {filename} s3://data-sandbox-zone-will-prod/upload/')

upload: ./cx.analise_sentimento_teste.csv to s3://data-sandbox-zone-will-prod/upload/cx.analise_sentimento_teste.csv


0

Análises:

- Cruzar com Detratores
- Cruzar com HMS v1
- Cruzar com segmentação LTV